In [20]:
import requests
import re
import pandas as pd

# списки обозначений верхов для языков
top_names_english = ["top", "apex", "crown", "peak", "pinnacle", "vertex", "summit","tip"]
top_names_french = ["apex", "cime", "haut", "pinacle", "sommet"] 
top_names_italian = ["cima", "apice", "picco", "pinnacolo", "vertice", "punta", "vetta"]
top_names_serbian = ["врх", "круна", "врхунац", "greben", "kresta"]

# списки ссылок по конструкциям
url_english_1_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=ukwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{0}%22+%5Bword%3D%22%28%3Fi%29{1}%22+%26+tag%3D%22N.*%22%5D&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&fsca_text.domain=&fsca_text.t2ld="
url_english_2_constr = """https://www.clarin.si/noske/wacs.cgi/first?corpname=ukwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%5Bword%3D"%28%3Fi%29{1}"+%26+tag%3D"N.*"%5D+"%28%3Fi%29of"+"%28%3Fi%29the"+"%28%3Fi%29{0}"&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&fsca_text.domain=&fsca_text.t2ld="""

url_french_1_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=frwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{1}%22+%22%28%3Fi%29du%22+%22%28%3Fi%29{0}%22&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&usesubcorp=&fsca_text.domain=&fsca_text.t2ld="
url_french_2_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=frwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{1}%22+%22%28%3Fi%29de%22+%22%28%3Fi%29la%22+%22%28%3Fi%29{0}%22&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&usesubcorp=&fsca_text.domain=&fsca_text.t2ld="
url_french_3_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=frwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{1}%22+%22%28%3Fi%29de%22+%22%28%3Fi%29l%27%22+%22%28%3Fi%29{0}%22&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&usesubcorp=&fsca_text.domain=&fsca_text.t2ld="

url_italian_1_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=itwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{1}%22+%22%28%3Fi%29dell%27%22+%22%28%3Fi%29{0}%22&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&usesubcorp=&fsca_text.domain=&fsca_text.t2ld="
url_italian_2_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=itwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{1}%22+%22%28%3Fi%29della%22+%22%28%3Fi%29{0}%22&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&usesubcorp=&fsca_text.domain=&fsca_text.t2ld="
url_italian_3_constr = "https://www.clarin.si/noske/wacs.cgi/first?corpname=itwac&reload=&iquery=&queryselector=cqlrow&lemma=&phrase=&word=&wpos=&char=&cql=%22%28%3Fi%29{1}%22+%22%28%3Fi%29del%22+%22%28%3Fi%29{0}%22&default_attr=word&fc_lemword_window_type=both&fc_lemword_wsize=5&fc_lemword=&fc_lemword_type=all&fc_pos_window_type=both&fc_pos_wsize=5&fc_pos_type=all&usesubcorp=&fsca_text.domain=&fsca_text.t2ld="

#url_serbian_1_constr = ""
#url_serbian_2_constr = ""


# Итальянский

In [3]:
df_italian = pd.read_csv("df_corpus/df_corpus_italian.csv", sep=',')
df_italian

,context,context_google,context_abby,протяженный,точечный,1D,2D,3D,нет увенч.,увенч.,1D жесткий,1D мягкий,cima,apice,picco,pinnacolo,vertice,punta,vetta
0,wall,parete,muro,1,0,1,0,0,0,0,1,0,2,0,0,0,0,0,0
1,gorge,gola,n,1,0,1,0,0,0,0,1,0,0,0,2,0,0,0,0
2,ridge,cresta,pettine,1,0,1,0,0,0,0,1,0,2,0,0,0,0,0,0
3,wave,onda,onda,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
4,waterfall,cascata,cascata,1,0,1,0,0,0,0,0,1,2,0,0,0,0,0,0
5,building,costruzione,edificio,1,0,0,1,0,0,0,0,0,2,2,0,0,2,0,0
6,skull,cranio,cranio,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,head,testa,testa,1,0,0,0,1,0,0,0,0,25,5,0,0,7,8,3
8,house,Casa,casa,1,0,0,0,1,0,0,0,0,8,0,0,0,107,52,0
9,arch,arco,arco,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Google translations

### 1 вид конструкции

In [15]:
# dell'
def query_wacky_it_1_constr(df):
    list_for_series = []
    context_names = df['context_google'].tolist()
    print(context_names)
    
    
    # необходимо выбрать список и ссылку для нужного языка
    url = url_italian_1_constr
    top_names = top_names_italian
    
    
    for top in top_names:
        list_for_series = []
        for context in context_names:
            print(context)
            # context = контекст, верх которого надо обозначить; top = обозначение верха
            url_formatted = url.format(context, top)
            # отправляю запрос на получение верха
            r = requests.get(url_formatted)
            corpora_reply = r.text
            # проверка на отсутствие вхождений
            error = re.search(r'<div id="error">', corpora_reply, re.DOTALL)
            if error is None:
                # ищу в запросе таблицу с ответами
                use_count_prep = re.search(r'<table.*?>.*?<\/table>', corpora_reply, re.DOTALL).group(0)
                # считаю количество ответов 
                use_count_prep = list(re.findall(r'<tr', use_count_prep, re.DOTALL))
                list_for_series.append(len(use_count_prep))
                print(list_for_series)
            else:
                list_for_series.append(0)
                print(list_for_series)
                
                
        column_name = top + '_dell'
        df[column_name] = pd.Series(list_for_series)
        print(list(df.columns))

### 2 вид конструкции

In [16]:
# della
def query_wacky_it_2_constr(df):
    list_for_series = []
    context_names = df['context_google'].tolist()
    print(context_names)
    
    
    # необходимо выбрать список и ссылку для нужного языка
    url = url_italian_2_constr
    top_names = top_names_italian
    
    
    for top in top_names:
        list_for_series = []
        for context in context_names:
            print(context)
            # context = контекст, верх которого надо обозначить; top = обозначение верха
            url_formatted = url.format(context, top)
            # отправляю запрос на получение верха
            r = requests.get(url_formatted)
            corpora_reply = r.text
            # проверка на отсутствие вхождений
            error = re.search(r'<div id="error">', corpora_reply, re.DOTALL)
            if error is None:
                # ищу в запросе таблицу с ответами
                use_count_prep = re.search(r'<table.*?>.*?<\/table>', corpora_reply, re.DOTALL).group(0)
                # считаю количество ответов 
                use_count_prep = list(re.findall(r'<tr', use_count_prep, re.DOTALL))
                list_for_series.append(len(use_count_prep))
                print(list_for_series)
            else:
                list_for_series.append(0)
                print(list_for_series)
                
                
        df[top] = pd.Series(list_for_series)
        print(list(df.columns))

### 3 вид конструкции

In [17]:
# del
def query_wacky_it_3_constr(df):
    list_for_series = []
    context_names = df['context_google'].tolist()
    print(context_names)
    
    
    # необходимо выбрать список и ссылку для нужного языка
    url = url_italian_3_constr
    top_names = top_names_italian
    
    
    for top in top_names:
        list_for_series = []
        for context in context_names:
            print(context)
            # context = контекст, верх которого надо обозначить; top = обозначение верха
            url_formatted = url.format(context, top)
            # отправляю запрос на получение верха
            r = requests.get(url_formatted)
            corpora_reply = r.text
            # проверка на отсутствие вхождений
            error = re.search(r'<div id="error">', corpora_reply, re.DOTALL)
            if error is None:
                # ищу в запросе таблицу с ответами
                use_count_prep = re.search(r'<table.*?>.*?<\/table>', corpora_reply, re.DOTALL).group(0)
                # считаю количество ответов 
                use_count_prep = list(re.findall(r'<tr', use_count_prep, re.DOTALL))
                list_for_series.append(len(use_count_prep))
                print(list_for_series)
            else:
                list_for_series.append(0)
                print(list_for_series)
                
                
        column_name = top + '_del'
        df[column_name] = pd.Series(list_for_series)
        print(list(df.columns))

### Запуск программ для создания колонок 

In [21]:
query_wacky_it_1_constr(df_italian) 
# query_wacky_it_2_constr(df_italian) done
# query_wacky_it_3_constr(df_italian) done

['parete', 'gola', 'cresta', 'onda', 'cascata', 'costruzione', 'cranio', 'testa', 'Casa', 'arco', 'vulcano', 'collina', 'montagna', 'tumulo', 'scogliera', 'roccia', 'casco', 'picco', 'pendenza', 'albero', 'Torre', 'piramide', 'iceberg', 'albero', 'polo', 'stelo', 'albero', 'tempio', 'Chiesa', 'campanile', 'Cattedrale', 'asparago']
parete
[0]
gola
[0, 0]
cresta
[0, 0, 0]
onda
[0, 0, 0, 1]
cascata
[0, 0, 0, 1, 0]
costruzione
[0, 0, 0, 1, 0, 0]
cranio
[0, 0, 0, 1, 0, 0, 0]
testa
[0, 0, 0, 1, 0, 0, 0, 0]
Casa
[0, 0, 0, 1, 0, 0, 0, 0, 0]
arco
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2]
vulcano
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0]
collina
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0]
montagna
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0]
tumulo
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
scogliera
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
roccia
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
casco
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]
picco
[0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
pendenza

In [22]:
df_italian

,context,context_google,context_abby,протяженный,точечный,1D,2D,3D,нет увенч.,увенч.,...,vertice_del,punta_del,vetta_del,cima_dell,apice_dell,picco_dell,pinnacolo_dell,vertice_dell,punta_dell,vetta_dell
0,wall,parete,muro,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,gorge,gola,n,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ridge,cresta,pettine,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,wave,onda,onda,1,0,1,0,0,0,0,...,0,0,0,1,1,3,0,1,0,0
4,waterfall,cascata,cascata,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,building,costruzione,edificio,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,skull,cranio,cranio,1,0,0,1,0,0,0,...,5,2,0,0,0,0,0,0,0,0
7,head,testa,testa,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,house,Casa,casa,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,arch,arco,arco,1,0,0,0,1,0,0,...,0,0,0,4,4,0,0,2,3,0


### Создание суммарных колонок по употреблениям

In [23]:
df_italian['cima_sum'] = df_italian[['cima', 'cima_dell', 'cima_del']].sum(axis=1)
df_italian['apice_sum'] = df_italian[['apice', 'apice_dell', 'apice_del']].sum(axis=1)
df_italian['picco_sum'] = df_italian[['picco', 'picco_dell', 'picco_del']].sum(axis=1)
df_italian['pinnacolo_sum'] = df_italian[['pinnacolo', 'pinnacolo_dell', 'pinnacolo_del']].sum(axis=1)
df_italian['vertice_sum'] = df_italian[['vertice', 'vertice_dell', 'vertice_del']].sum(axis=1)
df_italian['punta_sum'] = df_italian[['punta', 'punta_dell', 'punta_del']].sum(axis=1)
df_italian['vetta_sum'] = df_italian[['vetta', 'vetta_dell', 'vetta_del']].sum(axis=1)
df_italian

,context,context_google,context_abby,протяженный,точечный,1D,2D,3D,нет увенч.,увенч.,...,vertice_dell,punta_dell,vetta_dell,cima_sum,apice_sum,picco_sum,pinnacolo_sum,vertice_sum,punta_sum,vetta_sum
0,wall,parete,muro,1,0,1,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
1,gorge,gola,n,1,0,1,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,ridge,cresta,pettine,1,0,1,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,wave,onda,onda,1,0,1,0,0,0,0,...,1,0,0,1,1,3,0,1,0,0
4,waterfall,cascata,cascata,1,0,1,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
5,building,costruzione,edificio,1,0,0,1,0,0,0,...,0,0,0,2,2,0,0,2,0,0
6,skull,cranio,cranio,1,0,0,1,0,0,0,...,0,0,0,3,0,0,0,5,2,0
7,head,testa,testa,1,0,0,0,1,0,0,...,0,0,0,26,5,0,0,7,8,3
8,house,Casa,casa,1,0,0,0,1,0,0,...,0,0,0,8,0,0,0,107,52,0
9,arch,arco,arco,1,0,0,0,1,0,0,...,2,3,0,4,4,0,0,2,3,0


### Сохранение датафрейма языка в файл

In [24]:
df_italian.to_csv(r'df_corpus_big/df_corpus_italian_big.csv', index = False)

***

In [ ]:
# TODO
    #context_google_list = df['context_google'].tolist()
    #context_abby_list = df['context_abby'].tolist()
    #context_names = list(set(context_google_list + context_abby_list))
    
    # нужно добавить выкидывание дубликатов из таблицы ответов, но их обычно не более 3-5, большинство значений больше
    
    # нужно добавить поддержку подсчета через количество страниц,
    # 1стр = 1000, но такое встречается редко, поэтому потери информации не значительные
    
    # автоматизация создания суммарных колонок